## Import Packages

In [ ]:
from mstrio.connection import Connection
from mstrio.server import Cluster, ServerSettings
from mstrio.server import Environment, Project, compare_project_settings
from mstrio.users_and_groups.user_group import UserGroup, list_usergroups
from mstrio.users_and_groups.user import User, list_users
from mstrio.distribution_services import (ScheduleManager, EmailSubscription, SubscriptionManager,
                                          Content)
from mstrio.project_objects import list_dossiers
import pandas as pd

# PROD ENVIRONMENT
base_url_prod = "https://env-xxxxxx.customer.cloud.microstrategy.com/MicroStrategyLibrary/api/"
username_prod = "username"
password_prod = "passwd"

# DEV ENVIRONMENT
base_url_dev = "https://env-xxxxxx.customer.cloud.microstrategy.com/MicroStrategyLibrary/api/"
username_dev = "username"
password_dev = ""

mstr_tutorial_id = 'B7CA92F04B9FAE8D941C3E9B7E0CD754'

In [ ]:
#df = pd.DataFrame(quotes_final)
#df.to_csv("schedule.csv", index=False, encoding='utf-8', sep="#", quoting=csv.QUOTE_NONE, escapechar="\\", header=False)

import csv
# Define Data
RESULTS = ['apple','cherry','orange','pineapple','strawberry']

# Open File
resultFyle = open("output.csv",'w')

# Write data to file
for r in RESULTS:
    resultFyle.write(r + "\n")
resultFyle.close()

## Connect to Environment

In [ ]:
conn_prod = Connection(base_url_prod, username_prod, password_prod, login_mode=1, project_name="Tutorial_Prod")
env_prod = Environment(connection=conn_prod)
tutorial_prod = Project(conn_prod, name="Tutorial_Prod")


In [ ]:
conn_dev = Connection(base_url_dev, username_dev, password_dev, login_mode=1, project_name="Tutorial_Dev")
env_dev = Environment(connection=conn_dev)
tutorial_dev = Project(conn_dev, name="Tutorial_Dev")


***

***

# Server and Project

### See which services are running on the environment

In [ ]:
cluster = Cluster(connection=conn_dev)
node_name = 'env-239822laiouse1'

In [ ]:
cluster.nodes_topology()

### Stop service

In [ ]:
cluster.stop(service='Apache-Kafka', nodes=[node_name], login="mstr", passwd="8OT1CtHe21XJ")

### Start service

In [ ]:
cluster.start(service='Apache-Kafka', nodes=[node_name], login="mstr", passwd="8OT1CtHe21XJ")

### See which projects are on the environment

In [ ]:
cluster.list_projects()

### Unload project

In [ ]:
project_name = 'Consolidated Education Project'
cluster.unload_project(project_name=project_name)

### See loaded projects

In [ ]:
env_dev.list_loaded_projects()

### Load projects

In [ ]:
cluster.load_project(project_name=project_name)

### Export Project and Server Setting to JSON

In [ ]:
tutorial_dev = Project(conn)
tutorial_dev.settings.to_json("tutorial_dev_settings.json")

In [ ]:
env_dev = Cluster(conn)
env_dev.settings.to_json

In [ ]:
env_dev_settings = ServerSettings(conn_dev)
env_dev_settings.to_json("env_dev_settings.json")

### Compare project level settings of tutorial project in development and production environments

In [ ]:
compare_project_settings(projects=[tutorial_prod, tutorial_dev], show_diff_only=True)

### Compare server level settings of two environments

In [ ]:
# get settings of development environment
settings_dev = ServerSettings(connection=conn_dev)
setts_props_dev = settings_dev.list_properties()

# get settings of production environment
settings_prod = ServerSettings(connection=conn_prod)
setts_props_prod = settings_prod.list_properties()

# compare settings and save differences in a table of dictionaries
diff_table = []
for key_dev, value_dev in setts_props_dev.items():
    if value_dev != setts_props_prod[key_dev]:
        diff_table.append({'key': key_dev,
                           'value_dev': value_dev,
                           'value_prod': setts_props_prod[key_dev]})
pd.DataFrame.from_dict(diff_table)

### Update Server Settings of Dev environment to match Prod environment 

In [ ]:
update_dict = {}
for diff in diff_table:
    update_dict[diff['key']] = diff['value_prod']
    
if update_dict:
    settings_dev.alter(**update_dict)
    settings_dev.update()

### Export Prod Tutorial Project Settings into Dev Tutorial Project

In [ ]:
tutorial_prod.settings.to_csv("tutorial_prod_settings.csv")
tutorial_dev.settings.import_from("tutorial_prod_settings.csv")
tutorial_dev.settings.update()

In [ ]:
compare_project_settings(projects=[tutorial_prod, tutorial_dev], show_diff_only=True)

***

***

***

***

# User and Privileges

### Prepare data to connect to database

In [ ]:
import pyodbc # Python library to connect to database

driver = '{ODBC Driver 17 for SQL Server}'
server = '127.0.0.1'
database = 'TmpDB_MSTR'
uid = 'sa'
pwd = 'reallyStrongPwd123'

# prepare string which is used to connect to database
driver = 'DRIVER=' + driver
server = 'SERVER=' + server
database = 'DATABASE=' + database
uid = 'UID=' + uid
pwd = 'PWD=' + pwd
connection_string = ';'.join([driver, server, database, uid, pwd])

### Get newly hired employees from Database and create new user accounts for them. 

In [ ]:
query = "SELECT * FROM NEW_HIRE_EMPLOYEE WHERE EMPLOYEE_HIRE_DATE > '12-01-2020';"

with pyodbc.connect(connection_string) as db_conn:
    new_employees = pd.read_sql(query, db_conn)

new_users = []
for _, row in new_employees.iterrows():
    emp_username = row['EMPLOYEE_USER_NAME']
    emp_fullname = row['EMPLOYEE_LEGAL_NAME']
    emp_title = row['EMPLOYEE_TITLE']
    new_usr = User.create(connection=conn_dev, username=emp_username, full_name=emp_fullname,
                          trust_id=emp_username, require_new_password=False)
    new_users.append(new_usr)

### Add newly created users to Administrator group

In [ ]:
usergroup = UserGroup(connection=conn_dev, name='Administrator')
usergroup.add_users(users=new_users)

### Add email addresses with domain 'microstrategy.com' to newly created users

In [ ]:
new_users_with_emails = []
for usr in new_users:
    email_address = usr.username + '@microstrategy.com' 
    usr.add_address(name=usr.username, address=email_address)
    new_users_with_emails.append(usr)

### List empty usergroups

In [ ]:
all_usrgrps = list_usergroups(connection=conn_dev)
empty_usergroups = [usrgrp for usrgrp in all_usrgrps if not usrgrp.list_members()]

In [ ]:
for ug in empty_usergroups:
    print(ug.name)

### Delete empty usergroups

In [ ]:
ug = UserGroup(conn_dev, name="Web Viewers")
ug.delete(force=True)

### Disable few users and delete email addresses of them

In [ ]:
usernames = ['jsmith', 'rwilliams', 'mjohnson']
for username in usernames:
    usr = User(conn_dev, username=username)
    usr.alter(enabled=False)
    # remove all addresses from a given user
    if usr.addresses:
        for addr in usr.addresses:
            usr.remove_address(id=addr['id'])



***

***

***

***

***

***

# Subscriptions and Schedules

### Get list of schedules and its properties

In [ ]:
schedule_mngr = ScheduleManager(connection=conn_dev)
schedule_mngr.list_schedules()

### Create an email subscription that displays the list of departed users which still need to be removed from user groups

In [ ]:
lchase = User(conn_dev, name="Linda Chase")
new_sub = EmailSubscription.create(
    connection=conn_dev, name="Departed Users", email_subject='Remove departed users from usergroups',
    email_message='Users to remove are:', recipients=[lchase.id],
    schedules_ids='FF7BB3B311D501F0C00051916B98494F', project_name="Tutorial_Dev",
    contents=Content(id='FE72F33B11E7F54F000000802F31418D', type=Content.Type.DOSSIER, personalization=Content.Properties()))

    
    
#     project_name="MicroStrategy Tutorial",
    
    
#     contents=Content(id='ABC123ABC123ABC123ABC123ABC12345', type=Content.Type.REPORT),
#     schedules_ids=['ABC123ABC123ABC123ABC123ABC12345'], recipients=['ABC123ABC123ABC123ABC123ABC12345'])

In [ ]:
subs_mngr = SubscriptionManager(conn_dev, project_name='Tutorial_Dev')
subs_mngr.list_subscriptions(name="Departed Users")

In [ ]:
new_sub.alter(owner_id = lchase.id)

In [ ]:
for usr in new_users:
    new_sub.add_recipient(usr.id)

***

***

***

***

***

***

# Library and Dossier

### Get all dossiers on Server

In [ ]:
dss = list_dossiers(conn_dev)
dss

### Get info about a particular dossier

In [ ]:
dss[6].info

### Get all certified dossiers from admin library

In [ ]:
certified_dss = []
for ds in dss:
    if ds.certified_info['certified'] is True:
        certified_dss.append(ds)
    
certified_dss

### Publish certified dossiers in New Hires' Library

In [ ]:
for certified_ds in certified_dss:
    certified_ds.publish(recipients=new_users)

***

***

***

***

***

# Cleaning

### Change Server and Project Settings 

In [ ]:
env_dev_settings.import_from("env_dev_changed.json")
env_dev_settings.update()

In [ ]:
tutorial_dev.settings.import_from("tutorial_dev_changed.json")
tutorial_dev.settings.update()

### Restore users which were disabled/removed

In [ ]:

usrs = {'jsmith':'John Smith', 'rwilliams':'Robert Williams', 'mjohnson':'Micheal Johnson'}

for username, fullname in usrs.items():
    try:
        usr = User.create(conn_dev, username, fullname, require_new_password=False)
    except:
        usr = User(conn_dev, username=username)
        usr.alter(enable=True)
        
    if len(usr.addresses) == 0:
        email_address = usr.username + '@microstrategy.com'
        usr.add_address(name=usr.username, address=email_address)

### Delete user accounts created during execution of this script

In [ ]:
def delete_users(connection, fullname):
    usrs = list_users(connection=connection, name=fullname)
    for u in usrs:
        u.delete(force=True)

In [ ]:
fullnames = ['Linda Chase', 'Dylan Ayers', 'Nicky Macfarlane', 'Mark Ferry', 'Chandler Phillips', 'Bradley Daniels',
             'Brandon Rees']
for fullname in fullnames:
    delete_users(connection=conn_dev, fullname=fullname)

### Restore deleted usergroups

In [ ]:
ug_tmp_names = ["Web Viewers"]
for name in ug_tmp_names:
    UserGroup.create(conn_dev, name)

### Delete subscription

In [ ]:
new_sub.delete()

### Unpublish dossiers

In [ ]:
for certified_ds in certified_dss:
    certified_ds.unpublish(recipients=new_users)

### Close connections to environments

In [ ]:
conn_prod.close()

In [ ]:
conn_dev.close()